# Lab: Building a Self-Correcting RAG Chatbot with a UI

In this lab, we will build a complete, interactive application. The goal is to create a personal chatbot that answers questions based on provided documents (a PDF and a text file). This technique is a form of Retrieval-Augmented Generation (RAG).

We will then implement advanced agentic patterns like **Reflection** and **Self-Correction** from scratch, where a second AI agent evaluates the chatbot's answers and forces a retry if the quality is low. Finally, we'll wrap the entire application in a web-based user interface using Gradio.

### Setup: Your Personal Context

This chatbot is designed to represent a specific person. To personalize it, please do the following:

1.  **Replace `profile.pdf`**: In the `me` folder, replace the existing `profile.pdf` with a PDF of your own resume or LinkedIn profile.
2.  **Update `summary.txt`**: Edit the `me/summary.txt` file with a brief, third-person summary of your professional background.
3.  **Update `your_name`**: Change the value of the `your_name` variable in the code cell below.

In [ ]:
# === Imports ===
# PyPDF is used for reading text from PDF files.
# Gradio is used for creating the web UI.

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel

In [ ]:
# === Configuration and API Clients ===

load_dotenv(override=True)

# This is the primary client for the chatbot agent.
openai_client = OpenAI()

# We will use a separate client for the evaluator agent, in this case, Google's Gemini.
# This demonstrates how different models can be used for different tasks in an agentic system.
gemini_client = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/models"
)

### Step 1: Load Context from Files (Retrieval-Augmented Generation)

We will read the text from the PDF and the summary file. This content will be injected into our system prompt, giving the chatbot the necessary context to answer questions accurately.

In [ ]:
# === Personal Information ===
# IMPORTANT: Change this to your name.
your_name = "Atin Gupta"

In [ ]:
# Read the PDF file page by page and concatenate the text.
try:
    reader = PdfReader("me/profile.pdf")
    pdf_context = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            pdf_context += text
    print("Successfully loaded PDF context.")
except FileNotFoundError:
    print("Error: me/profile.pdf not found. Please add your profile PDF.")
    pdf_context = ""

In [ ]:
# Read the summary text file.
try:
    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary_context = f.read()
    print("Successfully loaded summary context.")
except FileNotFoundError:
    print("Error: me/summary.txt not found. Please add your summary file.")
    summary_context = ""

### Step 2: Define the Agent Personas (System Prompts)

We need two distinct personas: one for the chatbot agent and one for the evaluator agent. These are defined through detailed system prompts.

In [ ]:
# Persona for the main chatbot agent.
chatbot_system_prompt = f"""You are a helpful AI assistant acting as {your_name}. 
You are answering questions on {your_name}'s personal website. 
Your primary goal is to represent {your_name} faithfully and professionally, as if talking to a potential client, recruiter, or colleague.
Use the provided summary and profile information to answer questions about {your_name}'s career, skills, and experience.
If you do not know the answer to a question based on the context provided, it is better to say that you don't have that information.
--- CONTEXT --- 
## Summary:
{summary_context}

## Profile Details:
{pdf_context}
--- END CONTEXT ---
Now, please chat with the user, always staying in character as {your_name}.
"""

In [ ]:
# Persona for the evaluator agent.
evaluator_system_prompt = f"""You are a strict quality control evaluator. Your task is to decide if an AI agent's response is acceptable.
The agent is playing the role of {your_name} and must be professional, engaging, and factually consistent with the provided context.
Evaluate the agent's most recent response in the context of the user's question and the conversation history.
The response is UNACCEPTABLE if it is evasive, unprofessional, factually incorrect, or hallucinates information not present in the context.
The response is ACCEPTABLE if it is helpful, professional, and grounded in the provided information.
--- CONTEXT ---
## Summary:
{summary_context}

## Profile Details:
{pdf_context}
--- END CONTEXT ---
"""

### Step 3: Build the Self-Correction Workflow

This is the core of our agentic system. We will define three functions:
1.  `chat_agent`: The main chatbot that generates the initial response.
2.  `evaluation_agent`: The evaluator that checks the response quality.
3.  `rerun_agent`: The agent that re-attempts the answer if the first one was rejected, using the evaluator's feedback.

In [ ]:
# Pydantic models enforce a specific JSON structure for the LLM's output.
# This makes the output predictable and easy to parse.
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
def evaluation_agent(reply: str, message: str, history: list) -> Evaluation:
    """The agent that evaluates the chatbot's response."""
    print("--- Evaluating response... ---")
    
    # Create the prompt for the evaluator.
    evaluator_user_prompt = f"""An agent, acting as {your_name}, was asked a question by a User.
    Conversation History: {history}
    User's latest message: '{message}'
    Agent's response: '{reply}'
    Please evaluate the agent's response based on the criteria and context you were given.
    """
    
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt}
    ]
    
    # Use the `.parse()` method to automatically get structured JSON output.
    # This requires a Pydantic model (`Evaluation`) to define the schema.
    response = gemini_client.chat.completions.create(
        model="gemini-1.5-flash-latest:generateContent", 
        messages=messages, 
        response_format=Evaluation # This is a conceptual representation; actual API might differ.
    )
    # A more realistic implementation might involve a function call / tool call
    # For now, let's assume a compatible structured output feature.
    # As a fallback, we'll manually parse a JSON string if needed.
    try:
        parsed_response = Evaluation.model_validate_json(response.choices[0].message.content)
    except:
        # Fallback for models that don't perfectly adhere to the JSON schema
        print("Warning: Could not directly parse response. Assuming acceptable.")
        return Evaluation(is_acceptable=True, feedback="Could not parse evaluator response.")

    return parsed_response

In [ ]:
def rerun_agent(reply: str, message: str, history: list, feedback: str) -> str:
    """The agent that retries the answer after failure."""
    print("--- Rerunning with feedback... ---")
    
    # The system prompt is updated with the feedback from the evaluator.
    # This is the "Reflection" step, where the agent learns from its mistake.
    rerun_system_prompt = chatbot_system_prompt + f"""\n\n## Previous Attempt Failed
    Your previous answer was rejected by the quality control evaluator.
    Your attempted answer: '{reply}'
    Reason for rejection: '{feedback}'
    Please try again, taking this feedback into account to generate a better, more professional, and factually accurate response.
    """
    
    messages = [
        {"role": "system", "content": rerun_system_prompt}
    ] + history + [
        {"role": "user", "content": message}
    ]
    
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

### Step 4: Create the Main Chat Interface

This final function orchestrates the entire workflow. It takes the user's message, gets a response, sends it to the evaluator, and triggers a rerun if necessary. This function will be connected to the Gradio UI.

In [ ]:
def chat_workflow(message: str, history: list):
    """The main workflow that connects all the agents."""
    print(f"\n--- New Request: {message} ---")
    
    # Gradio's history format can sometimes include extra data.
    # This line cleans it to ensure compatibility with the OpenAI API.
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    # === Initial Response ===
    print("--- Generating initial response... ---")
    messages = [
        {"role": "system", "content": chatbot_system_prompt}
    ] + history + [
        {"role": "user", "content": message}
    ]
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content
    
    # === Evaluation ===
    evaluation = evaluation_agent(reply, message, history)
    
    # === Self-Correction ===
    if evaluation.is_acceptable:
        print("\n*** PASSED EVALUATION ***")
        return reply
    else:
        print(f"\n*** FAILED EVALUATION ***\nFeedback: {evaluation.feedback}")
        final_reply = rerun_agent(reply, message, history, evaluation.feedback)
        return final_reply

In [ ]:
# Launch the Gradio web interface.
# This creates a public URL where you can interact with your chatbot.
gr.ChatInterface(
    chat_workflow, 
    title=f"{your_name}'s Personal AI Assistant",
    description="Ask me questions about my professional background, skills, and experience.",
    examples=["What is your experience with Python?", "Can you summarize your key skills?", "Tell me about your most recent role."]
).launch()